# Predicting Football Match Winners

In this notebook, we'll use machine learning to predict who will win EPL football matches. We'll:
1. Load and clean the match data
2. Create features for our model
3. Train a Random Forest classifier
4. Evaluate and improve our predictions

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score

## Load the Data

In [ ]:
matches = pd.read_csv("matches.csv", index_col=0)

In [ ]:
matches.head()

In [ ]:
matches.shape

## Data Preprocessing

We need to:
1. Convert the result column to a binary target (win = 1, loss/draw = 0)
2. Convert date to datetime
3. Create a venue code (Home = 1, Away = 0)
4. Create opponent code

In [ ]:
matches["date"] = pd.to_datetime(matches["date"])
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek

In [ ]:
matches["target"] = (matches["result"] == "W").astype("int")

In [ ]:
matches.head()

## Initial Model Training

Let's start with a simple Random Forest model using basic features.

In [ ]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [ ]:
train = matches[matches["date"] < '2022-01-01']
test = matches[matches["date"] > '2022-01-01']

In [ ]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [ ]:
rf.fit(train[predictors], train["target"])

In [ ]:
preds = rf.predict(test[predictors])

In [ ]:
acc = accuracy_score(test["target"], preds)
print(f"Accuracy: {acc}")

In [ ]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

## Precision Score

For this problem, precision is more important than accuracy. We want to know: when we predict a win, how often is it actually a win?

In [ ]:
precision_score(test["target"], preds)

## Improve Predictions with Rolling Averages

We'll add rolling averages of team statistics from previous matches to improve our predictions.

In [ ]:
grouped_matches = matches.groupby("team")

In [ ]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [ ]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

In [ ]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [ ]:
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])

## Make Predictions Function

Create a function to make predictions and calculate precision.

In [ ]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [ ]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [ ]:
print(f"Precision with rolling averages: {precision}")

## Merge Predictions with Match Details

In [ ]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [ ]:
combined.head(10)

## Merge Home and Away Predictions

We can improve our model by considering both home and away team predictions for the same match.

In [ ]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
}
mapping = MissingDict(**map_values)

In [ ]:
combined["new_team"] = combined["team"].map(mapping)

In [ ]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [ ]:
merged.head()

## Analyze Combined Predictions

When both our model and the opponent prediction agree, how accurate are we?

In [ ]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()

## Summary

Our model achieves reasonable precision in predicting football match winners:
- Base model with simple features: ~50% precision
- With rolling averages: ~60% precision
- When combining home/away predictions: even higher confidence

The model performs best when:
- Predicting wins for strong home teams
- Using rolling averages of recent form
- Combining predictions from both teams in a match